# Challenge3
    - LCEL 체인 구성
    - 항상 세개의 이모티콘으로 답장하도록 예제 적용(Fewshots)
    - 메모리 사용(LCEL Memory)

In [50]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory




llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-4o-mini-2024-07-18",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

#Memory
'''
ConversationSummaryMemory: llm사용하여 대화를 요약하며 저장
ConversationBufferMemory: 단순히 이전 대화 내용 전체를 저장
ConversationSummaryBufferMemory: 저장하다가 limit에 다다르면 오래된 메세지들 요약
'''
#메모리 정의
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,   
    return_messages=True #chat 형식으로 저장
)


#----------- example
'''
1. example, example_prompt: 예제들 생성
2. prompt
    - system: 시스템 설정
    - human: 질문
    - memory: 대답들 저장
        - MessagesPlaceholder: 채팅형식으로 질문과 답변들 저장
    - example_prompt: 답변은 예제프롬프트처럼
'''

#Fewshot
examples = [
    {"question": "Top Gun", "answer": "🛩️👨‍✈️🔥"},
    {"question": "Godfather", "answer": "👨‍👨‍👦🔫🍝"},
    {"question": "Spiderman", "answer": "🥽🦺🥇"},
]
example_prompt = ChatPromptTemplate.from_messages([
    ('human', '{question}'),
    ('ai', '{answer}')
])
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

prompt = ChatPromptTemplate.from_messages([
    (
        'system',
        '''
        You are a critic who can creatively express the content of a movie. When the name of the movie comes in, you have to express it with three emojis. Must be displayed in 3 images only
        '''
    ),
    #question에 대한 답변은 chat_history안에 계속 저장됨
    MessagesPlaceholder(variable_name="history"), #채팅형식으로 메모리 저장
    example_prompt,
    ('human', '{question}')
])


'''
처음: question, load_memory(_)=빈값이 prompt로 전달 > llm 거쳐서 > answer도출
memory.save_context를 통해 question과 도출된 aswer를 저장
'''

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    ) 
    print(result)

In [51]:
invoke_chain('범죄도시')

👮‍♂️🔍💥content='👮\u200d♂️🔍💥' response_metadata={'finish_reason': 'stop'} id='run-e6f9d7c2-2b27-4aba-b7fe-1c17202d6591-0'


In [52]:
invoke_chain('aqua man')

🌊🧜‍♂️⚓content='🌊🧜\u200d♂️⚓' response_metadata={'finish_reason': 'stop'} id='run-7251b3f4-90bb-460e-88df-3444515dbd36-0'


In [54]:
invoke_chain('avatar')

🌌🧚‍♂️🌿content='🌌🧚\u200d♂️🌿' response_metadata={'finish_reason': 'stop'} id='run-6aa6cb72-9349-44d4-b607-920d45853212-0'


In [55]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='범죄도시'),
  AIMessage(content='👮\u200d♂️🔍💥'),
  HumanMessage(content='aqua man'),
  AIMessage(content='🌊🧜\u200d♂️⚓'),
  HumanMessage(content='avatar'),
  AIMessage(content='🌌🧚\u200d♂️🌿')]}